# Dangerous asteroids!

In [ ]:
%load_ext lab_black

import requests
import json
import pandas as pd
import cufflinks
import datetime as dt
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pymc3 as pm
import theano.tensor as tt

cufflinks.go_offline(False)

In [ ]:
datas = {}

for i in tqdm(
    pd.date_range(
        start=dt.date.today() - dt.timedelta(days=200),
        end=dt.date.today() - dt.timedelta(days=1),
        freq="W",
    )
):
    datas.update(
        json.loads(
            requests.get(
                "https://api.nasa.gov/neo/rest/v1/feed",
                {"start_date": str(i)[:10], "api_key": "DEMO_KEY"},
            ).content
        )["near_earth_objects"]
    )

In [ ]:
df = pd.DataFrame(
    {
        key: {
            "dangerous": len(
                [_ for i in value if i["is_potentially_hazardous_asteroid"]]
            ),
            "total": len(value),
        }
        for (key, value) in datas.items()
    }
).T
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)

In [ ]:
df[["dangerous"]].iplot(title="Number of asteroids coming close to Earth, by day")

In [ ]:
with pm.Model() as model:
    alpha = 1.0 / df.dangerous.mean()
    lambda_ = pm.Exponential("lambda", alpha)

In [ ]:
with model:
    observation = pm.Poisson("obs", lambda_, observed=df.dangerous)

In [ ]:
with model:
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000, step=step)

In [ ]:
x = np.linspace(trace["lambda"].min(), trace["lambda"].max(), 1000)

pd.Series(kde(trace["lambda"]).evaluate(x), index=x).iplot(
    title="KDE plot of parameter lambda"
)

## Probability of observing 5 or more dangerous asteroids in a day? Less  than 1%!

In [ ]:
1 - poisson(trace["lambda"].mean()).cdf(4)